# Plain-Text CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import os  # operating system functions
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices
import matplotlib.pyplot as plt  # plotting
from sklearn.metrics import mean_squared_error  # MSE error metric
from analysis_functions import (evaluate_forecasts, train_tf_model, pred_tf_model,
                                train_pt_model, pred_pt_model)  # own functions
from preproc_functions import list_combine  # own functions
from neural_nets import UNet, MNet  # own defined neural nets as classes

# pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn

# keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adam, Adagrad, SGD

Using TensorFlow backend.


### datasets

#### univariate training

In [3]:
train_Xu = np.load(dir_data+'train_Xu.npy')
train_yu = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_Xu.shape)
print(train_yu.shape)

(988, 14, 1)
(988, 7)


\>980 Observations for X days for 1 Variable!

#### multivariate training

In [5]:
train_Xm = np.load(dir_data+'train_Xm.npy')
train_ym = np.load(dir_data+'train_ym.npy')

In [6]:
print(train_Xm.shape)
print(train_ym.shape)

(988, 14, 8)
(988, 7)


\>980 Observations for X days for 8 Variables!

#### test set

In [7]:
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]
test_Xm = np.load(dir_data+'test_Xs.npy')
test_y = np.load(dir_data+'test_ys.npy')

In [8]:
print(test_Xu.shape)
print(test_Xm.shape)
print(test_y.shape)

(59, 14, 1)
(59, 14, 8)
(59, 7)


## Modeling

### Options

In [9]:
# net hyper parameters
# univariate specific
epochs_u = 20
batch_size_u = 4
# multivariate specific
epochs_m = 70
batch_size_m = 16
# general
lr = 0.001
n_filters = 16
n_linear = 10

In [10]:
# dimensions
n_timesteps = train_Xu.shape[1] 
n_features_u = train_Xu.shape[2]
n_features_m = train_Xm.shape[2]
n_outputs = train_yu.shape[1] 

### Tensorflow

In [11]:
# build univariate model
model_tfu = Sequential()
model_tfu.add(Conv1D(filters=n_filters, kernel_size=3, activation='relu', 
                     input_shape=(n_timesteps, n_features_u)))
model_tfu.add(MaxPooling1D(pool_size=2))
model_tfu.add(Flatten())
model_tfu.add(Dense(n_linear, activation='relu'))
model_tfu.add(Dense(n_outputs))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
# build multivariate model
model_tfm = Sequential()
model_tfm.add(Conv1D(filters=n_filters*2, kernel_size=3, activation='relu', 
                     input_shape=(n_timesteps, n_features_m)))
if n_timesteps >= 14:
    model_tfm.add(Conv1D(filters=n_filters*2, kernel_size=3, activation='relu'))
    model_tfm.add(MaxPooling1D(pool_size=2))
model_tfm.add(Conv1D(filters=n_filters, kernel_size=3, activation='relu'))
model_tfm.add(MaxPooling1D(pool_size=2))
model_tfm.add(Flatten())
model_tfm.add(Dense(n_linear*10, activation='relu'))
model_tfm.add(Dense(n_outputs))

#### Univariate

In [13]:
model_tfu, opt_tfu = train_tf_model(train_Xu, train_yu, model_tfu,
                                   batch_size_u, epochs_u, lr)
preds_tfu, rmse_tfu = pred_tf_model(test_Xu, test_y, model_tfu)

Instructions for updating:
Use tf.cast instead.
Overall RMSE for TF model: 403.204745822753!


#### Multivariate

In [14]:
model_tfm, opt_tfm = train_tf_model(train_Xm, train_ym, model_tfm,
                                    batch_size_u, epochs_u, lr)
preds_tfm, rmse_tfm = pred_tf_model(test_Xm, test_y, model_tfm)

Overall RMSE for TF model: 422.13744692506356!


### Pytorch

#### Univariate

In [15]:
model_ptu = UNet(n_features_u, n_filters, n_timesteps, n_linear, n_outputs)

In [16]:
model_ptu, opt_ptu = train_pt_model(train_Xu, train_yu, model_ptu,
                                    batch_size_u, epochs_u, lr)
preds_ptu, rmse_ptu = pred_pt_model(test_Xu, test_y, model_ptu)

Overall RMSE for PT model: 398.4623874918848!


#### Multivariate

In [17]:
model_ptm = MNet(n_features_m, n_filters, n_timesteps, n_linear, n_outputs)

In [18]:
model_ptm, opt_ptm = train_pt_model(train_Xm, train_ym, model_ptm,
                                    batch_size_m, epochs_m, lr)
preds_ptm, rmse_ptm = pred_pt_model(test_Xm, test_y, model_ptm)

Overall RMSE for PT model: 398.0896687083875!


### Saving

#### rmse & predictions

In [19]:
l1 = ['rmse', 'preds']
l2 = ['_tf', '_pt']
l3 = ['m', 'u']

In [20]:
file_names = list_combine(list_combine(l1,l2), l3)

In [21]:
for item in file_names[:4]:
    np.save(dir_data+'rmse/'+item, eval(item)[1])

In [22]:
for item in file_names[4:]:
    np.save(dir_data+'predictions/'+item, eval(item)[1])

#### trained models

In [23]:
# save in dedicated location
if not os.path.exists(dir_data+'models/'):
    os.makedirs(dir_data+'models/')

In [24]:
# tensorflow
model_tfu.save(dir_data+'models/model_tfu.h5')
model_tfm.save(dir_data+'models/model_tfm.h5')

In [25]:
# pytorch
torch.save(model_ptu, dir_data+'models/model_ptu.pt')
torch.save(model_ptm, dir_data+'models/model_ptm.pt')